In [5]:
!pwd

/c/Users/tsitsimis/Documents/projects/safage-stack/_notebooks


In [6]:
!ls ../

_notebooks  examples  requirements.dev.txt  requirements.txt  safage


In [8]:
import os
import json
from typing import List
from collections import defaultdict

from sqlalchemy import create_engine
import pandas as pd

In [10]:
os.listdir("../examples/create-sql-tables/great_expectations/expectations")

['.ge_store_backend_id', 'passenger']

In [12]:
import pathlib

list(pathlib.Path("../examples/create-sql-tables/great_expectations/expectations").iterdir())

[PosixPath('../examples/create-sql-tables/great_expectations/expectations/.ge_store_backend_id'),
 PosixPath('../examples/create-sql-tables/great_expectations/expectations/passenger')]

In [7]:
suite = json.load(open("../examples/create-sql-tables/great_expectations/expectations/passenger/basic.json", "r"))
suite

{'data_asset_type': None,
 'expectation_suite_name': 'passenger.basic',
 'expectations': [{'expectation_type': 'expect_column_values_to_be_of_type',
   'kwargs': {'column': 'passenger_id', 'type_': 'int'}},
  {'expectation_type': 'expect_column_values_to_be_unique',
   'kwargs': {'column': 'passenger_id'}},
  {'expectation_type': 'expect_column_values_to_be_of_type',
   'kwargs': {'column': 'survived', 'type_': 'int'}},
  {'expectation_type': 'expect_column_values_to_be_in_set',
   'kwargs': {'column': 'survived', 'value_set': [0, 1]}},
  {'expectation_type': 'expect_column_values_to_be_of_type',
   'kwargs': {'column': 'pclass', 'type_': 'int'}},
  {'expectation_type': 'expect_column_values_to_be_in_set',
   'kwargs': {'column': 'pclass', 'value_set': [1, 2, 3]}},
  {'expectation_type': 'expect_column_values_to_be_of_type',
   'kwargs': {'column': 'name', 'type_': 'str'}},
  {'expectation_type': 'expect_column_value_lengths_to_be_between',
   'kwargs': {'column': 'name', 'max_value': 

In [11]:
engine = create_engine("sqlite://")

In [12]:
from sqlalchemy import Table, Column, Integer, String, Float, ForeignKey

In [13]:
from sqlalchemy import MetaData

metadata = MetaData()

### Read pandas dataframe

In [22]:
passengers = pd.read_csv("../../../data/passengers.csv")

In [23]:
passengers.head()

,passenger_id,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Create table based on GE suite

In [24]:
table_name = suite["expectation_suite_name"].split(".")[0]
table_name

'passenger'

In [84]:
def get_column_names(expectations: dict) -> List[str]:
    column_expectations = filter(lambda x: x["expectation_type"].startswith("expect_column"), expectations)
    column_names = map(lambda x: x["kwargs"]["column"], column_expectations)
    return list(set(column_names))


def get_primary_key_columns(column_names: List[str]) -> List[str]:
    pk_columns = filter(lambda x: x.endswith("_id"), column_names)
    return list(pk_columns)


def ge_to_sqla_types(ge_type: str, **kwargs):
    if ge_type == "str":
        return String(kwargs["length"])
    if ge_type == "int":
        return Integer
    if ge_type == "float":
        return Float


def get_column_definition(expectations: dict) -> List:
    """
    Builds a dictionary containing the column definition for SQLAlchemy. The dict has the form
    {
        "column_name": {"type": <sql_type>, "primary_key": <bool>}
    }
    """
    
    column_names = get_column_names(expectations)
    
    column_definitions = defaultdict(dict)
    
    for column_name in column_names:
        # type
        all_columns_type_expectations = filter(lambda x: x["expectation_type"] == "expect_column_values_to_be_of_type", expectations)
        column_type_expectations = filter(lambda x: x["kwargs"]["column"] == column_name, all_columns_type_expectations)
        ge_type = list(column_type_expectations)[0]["kwargs"]["type_"]
        
        if ge_type == "str":
            all_columns_length_expectations = filter(lambda x: x["expectation_type"] == "expect_column_value_lengths_to_be_between", expectations)
            column_length_expectations = filter(lambda x: x["kwargs"]["column"] == column_name, all_columns_length_expectations)
            length = list(column_length_expectations)[0]["kwargs"]["max_value"]
            
            column_definitions[column_name]["type"] = String(length)
        else:
            column_definitions[column_name]["type"] = ge_to_sqla_types(ge_type)
            
        if column_name.endswith("_id"):
            column_definitions[column_name]["primary_key"] = True
        
    return column_definitions

In [85]:
column_definition = get_column_definition(suite["expectations"])
column_definition

defaultdict(dict,
            {'sex': {'type': String(length=10)},
             'name': {'type': String(length=100)},
             'pclass': {'type': sqlalchemy.sql.sqltypes.Integer},
             'age': {'type': sqlalchemy.sql.sqltypes.Integer},
             'passenger_id': {'type': sqlalchemy.sql.sqltypes.Integer,
              'primary_key': True},
             'survived': {'type': sqlalchemy.sql.sqltypes.Integer}})

In [86]:
Table(
    table_name,
    metadata,

    *[
        Column(column_name, definition["type"], primary_key=("primary_key" in definition.keys())) \
        for column_name, definition in column_definition.items()
    ],
    extend_existing=True
    
)

Table('passenger', MetaData(bind=None), Column('sex', String(length=10), table=<passenger>), Column('name', String(length=100), table=<passenger>), Column('pclass', Integer(), table=<passenger>), Column('age', Integer(), table=<passenger>), Column('passenger_id', Integer(), table=<passenger>, primary_key=True, nullable=False), Column('survived', Integer(), table=<passenger>), schema=None)

In [92]:
metadata.tables.keys()

dict_keys(['passenger'])